In [860]:
import os,gzip,itertools,csv,re,sys,io, Bio
import os.path
import urllib.request
import pandas as pd
import numpy as np
import urllib
import csv
import fastaparser as fp
import Bio
from Bio import SeqIO
from Bio import Entrez
from ftplib import FTP
import warnings
from pathlib import Path
import pickle

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [948]:
dat = pd.read_csv('ensemble_metazoa.csv')


#with open('ensemble_metazoa.csv', "r", newline='') as csvfile:
#    read = csv.reader(csvfile, delimiter=',')
#    for row in read:
#        if row[2] != "Crustacea" or "Hemiptera" or "Diptera" or "Coleoptera" or "Hymenoptera" or "Lepidoptera" or "Chelicerata" or "Phthiraptera" or "Isoptera":
            
#dat
#dat.Classification

insecta = dat[(dat.Accession.isin(["GCA_000142985.2", "GCA_002204515.1", "GCA_001444175.1", "GCA_003254395.2", "GCA_000143395.2", "GCA_000775305.1", "GCA_000188095.2", "GCA_000214255.1", "GCA_000151625.1", "GCA_000648675.1", "GCA_000235995.2", "GCA_000313835.1", "GCA_000208615.1", "GCA_002892825.2", "GCA_003675905.1", "GCA_001187945.1", "GCA_000716385.1", "GCA_000371365.1", "GCA_009193385.2", "GCA_000006295.1", "GCA_000181055.3", "GCA_000828355.1", "GCA_000188075.2", "GCA_000611955.2", "GCA_001015335.1", "GCA_002237135.1", "GCA_000239435.1", "GCA_007210705.1", "GCA_000002335.3", "GCA_002443255.1", "GCA_000696155.1"]))]

insecta[["Name", "Classification", "Taxon ID", "Assembly", "Accession"]]


,Name,Classification,Taxon ID,Assembly,Accession
0,Acyrthosiphon pisum,Hemiptera,7029,Acyr_2.0,GCA_000142985.2
2,Aedes aegypti (LVP_AGWG),Diptera,7159,AaegL5,GCA_002204515.1
3,Aedes albopictus,Diptera,7160,AaloF1,GCA_001444175.1
28,Apis mellifera (DH4),Hymenoptera,7460,Amel_HAv3.1,GCA_003254395.2
29,Atta cephalotes,Hymenoptera,12957,Attacep1.0,GCA_000143395.2
30,Belgica antarctica,Diptera,315563,ASM77530v1,GCA_000775305.1
32,Bombus impatiens,Hymenoptera,132113,BIMP_2.0,GCA_000188095.2
33,Bombus terrestris,Hymenoptera,30195,Bter_1.0,GCA_000214255.1
34,Bombyx mori,Lepidoptera,7091,ASM15162v1,GCA_000151625.1
43,Cimex lectularius,Hemiptera,79782,ClecH1,GCA_000648675.1


In [624]:
# this is code which will parse FASTA files
# define what a header looks like in FASTA format
def isheader(line):
    return line[0] == '>'

def aspairs(f):
    seq_id = ''
    sequence = ''
    for header,group in itertools.groupby(f, isheader):
        if header:
            line = next(group)
            seq_id = line[1:].split()[0]
        else:
            sequence = ''.join(line.strip() for line in group)
            yield seq_id, sequence
            
            
            

In [911]:
# Get names
GFFdirnames = os.listdir('GFF3s')

#Loop through files
GFFdir = os.scandir('GFF3s')
#print(GFFdirnames)

## Obtain genome stats
GFFdat = []
for f in GFFdir:
    with gzip.open(f, "rt") as fh:
        mygff = csv.reader(fh, delimiter="\t")
        count = 0
        gene_len = 0
        CDS_count = 0
        CDS_len = 0
        exon_count = 0
        exon_len = 0
        intron_count = 0
        intron_len = 0
        genome_size=0
        for row in mygff:
            if row[0].startswith("#"):
                continue
            #print(row)
            if row[2] == "gene":
                count += 1
                total_gene_len = int(row[4])-int(row[3])
                gene_len += total_gene_len
            if row[2] == "CDS":
                CDS_count += 1
                total_CDS_len =int(row[4])-int(row[3])
                CDS_len +=  total_CDS_len
            if row[2] == "exon":
                exon_count += 1
                total_exon_len =int(row[4])-int(row[3])
                exon_len +=  total_exon_len
                    
        mylist=list([Path(f).stem, count, gene_len, exon_count, exon_len, CDS_count, CDS_len])

    GFFdat.append(mylist)
print(GFFdat)
            #df = pd.DataFrame(columns = ["Name","Num_of_Genes","Gene_Length","Num_of_Exons","Exon_Length","Num_of_CDS","CDS_Length"])
            #df.append(mylist)
            #print(df)
            #print ("The number of genes in {} is {}".format(Path(f).stem, count),".")
            #print ("Total length of genes in {} is {} bp ({} kb)".format(Path(f).stem, gene_len,gene_len/1000), ".")
            #print ("The number of CDS in {} is {}".format(Path(f).stem,CDS_count),".")
            #print ("Total length of CDS in {} is {} bp ({} kb)".format(Path(f).stem, CDS_len,CDS_len/1000), ".")
            #print(exon_count)
            #print(exon_len)


/rhome/hchu036/.conda/envs/GEN220/lib/python3.7/site-packages/ipykernel_launcher.py:5: ResourceWarning: unclosed scandir iterator <posix.ScandirIterator object at 0x7f4d1ccfa960>
  """


[['Pediculus_humanus.PhumU2.49.gff3', 10785, 34055353, 70308, 16766929, 69317, 16601635], ['Atta_cephalotes.Attacep1.0.49.gff3', 10491, 123030673, 75044, 19183647, 72364, 18204745], ['Stegodyphus_mimosarum.Stegodyphus_mimosarum_v1.49.gff3', 27135, 603727801, 119586, 25494976, 117433, 23073047], ['Musca_domestica.MdomA1.49.gff3', 14402, 217154639, 109380, 48263046, 98017, 37017391], ['Teleopsis_dalmanni.Tel_dalmanni_2A_v1.0.49.gff3', 16571, 77374125, 113851, 25774923, 106374, 21665568], ['Heliconius_melpomene.Hmel1.49.gff3', 12669, 107349118, 97113, 27146437, 84599, 17386742], ['Tetranychus_urticae.ASM23943v1.49.gff3', 17671, 48177832, 80981, 32445136, 62549, 20418729], ['Aedes_albopictus.AaloF1.49.gff3', 17535, 163385473, 58951, 37928066, 58188, 24159585], ['Rhodnius_prolixus.RproC3.49.gff3', 15068, 122260425, 90431, 16967587, 87833, 15996083], ['Apis_mellifera.Amel_HAv3.1.49.gff3', 9944, 167799308, 267978, 98462255, 226454, 57169657], ['Melitaea_cinxia.MelCinx1.0.49.gff3', 16639, 1570

In [913]:
df = pd.DataFrame(GFFdat, columns = ["Name","Num_of_Genes","Gene_Length","Num_of_Exons","Exon_Length","Num_of_CDS","CDS_Length"]) 
df

,Name,Num_of_Genes,Gene_Length,Num_of_Exons,Exon_Length,Num_of_CDS,CDS_Length
0,Pediculus_humanus.PhumU2.49.gff3,10785,34055353,70308,16766929,69317,16601635
1,Atta_cephalotes.Attacep1.0.49.gff3,10491,123030673,75044,19183647,72364,18204745
2,Stegodyphus_mimosarum.Stegodyphus_mimosarum_v1...,27135,603727801,119586,25494976,117433,23073047
3,Musca_domestica.MdomA1.49.gff3,14402,217154639,109380,48263046,98017,37017391
4,Teleopsis_dalmanni.Tel_dalmanni_2A_v1.0.49.gff3,16571,77374125,113851,25774923,106374,21665568
5,Heliconius_melpomene.Hmel1.49.gff3,12669,107349118,97113,27146437,84599,17386742
6,Tetranychus_urticae.ASM23943v1.49.gff3,17671,48177832,80981,32445136,62549,20418729
7,Aedes_albopictus.AaloF1.49.gff3,17535,163385473,58951,37928066,58188,24159585
8,Rhodnius_prolixus.RproC3.49.gff3,15068,122260425,90431,16967587,87833,15996083
9,Apis_mellifera.Amel_HAv3.1.49.gff3,9944,167799308,267978,98462255,226454,57169657
